In [87]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
  
driver = webdriver.Chrome("C:\\Drivers\\chrome\\chromedriver.exe")
  
driver.get("https://linkedin.com/uas/login")
  
time.sleep(1)

username = driver.find_element_by_id("username")
  
username.send_keys("thesecondbit2@gmail.com")
  
pword = driver.find_element_by_id("password")
  
pword.send_keys("9Q1s4u$SD#Vy")
  
driver.find_element_by_xpath("//button[@type='submit']").click()

C:\Users\nope\AppData\Local\Temp\ipykernel_11044\2525572240.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(".\\chrome\\chromedriver.exe")


WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [77]:
profile_url = "https://www.linkedin.com/in/suhailahmedvelorum/"
  
driver.get(profile_url)

In [78]:
start = time.time()
  
# will be used in the while loop
initialScroll = 0
finalScroll = 1000
  
while True:
    driver.execute_script(f"window.scrollTo({initialScroll},{finalScroll})")
   
    initialScroll = finalScroll
    finalScroll += 1000
  
    time.sleep(1)
  
    end = time.time()
 
    if round(end - start) > 5:
        break

In [79]:
src = driver.page_source
  
# Now using beautiful soup
soup = BeautifulSoup(src, 'lxml')

In [80]:
sections = soup.find_all('section')
about_section = -1
for idx, section in enumerate(sections):
    if len(section.find_all('div', {'id': "about"})) > 0:
        about_section = section
about = about_section.find_all('span', {'class': "visually-hidden"})[-1]

In [81]:
import pickle

In [82]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import re
import pickle
import numpy as np
from ipywidgets import widgets, interact

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

# python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")


def tokeniser(sentence):
 
    sentence = re.sub("[]|||[]", " ", sentence)

    sentence = re.sub("/r/[0-9A-Za-z]", "", sentence)

    MBTI_types = ['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
              'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ',
              'MBTI']
    MBTI_types = [ti.lower() for ti in MBTI_types] + [ti.lower() + 's' for ti in MBTI_types]

    tokens = nlp(sentence)

    tokens = [ti for ti in tokens if ti.lower_ not in STOP_WORDS]
    tokens = [ti for ti in tokens if not ti.is_space]
    tokens = [ti for ti in tokens if not ti.is_punct]
    tokens = [ti for ti in tokens if not ti.like_num]
    tokens = [ti for ti in tokens if not ti.like_url]
    tokens = [ti for ti in tokens if not ti.like_email]
    tokens = [ti for ti in tokens if ti.lower_ not in MBTI_types]


    # lemmatize
    tokens = [ti.lemma_ for ti in tokens if ti.lemma_ not in STOP_WORDS]
    tokens = [ti for ti in tokens if len(ti) > 1]

    return tokens


model_save_location = "./models/"

with open(model_save_location + 'cv.pickle', 'rb') as f:
    cv = pickle.load(f)
    
with open(model_save_location + 'idf_transformer.pickle', 'rb') as f:
    idf_transformer = pickle.load(f)
    
# loading the pickle files with the classifiers
with open(model_save_location + 'LR_clf_IE_kaggle.pickle', 'rb') as f:
    lr_ie = pickle.load(f)
with open(model_save_location + 'LR_clf_JP_kaggle.pickle', 'rb') as f:
    lr_jp = pickle.load(f)
with open(model_save_location + 'LR_clf_NS_kaggle.pickle', 'rb') as f:
    lr_ns = pickle.load(f)
with open(model_save_location + 'LR_clf_TF_kaggle.pickle', 'rb') as f:
    lr_tf = pickle.load(f)


def eval_string(my_post, show_graph=False):
    c = cv.transform([tokeniser(my_post)])
    x = idf_transformer.transform(c)
    
    ie = lr_ie.predict_proba(x).flatten()
    ns = lr_ns.predict_proba(x).flatten()
    tf = lr_tf.predict_proba(x).flatten()
    jp = lr_jp.predict_proba(x).flatten()
    
    probs = np.vstack([ie, ns, tf, jp])
    
    names = ["Introversion - Extroversion", 
             "Intuiting - Sensing", 
             "Thinking - Feeling", 
             "Judging - Perceiving"]
    
    for i, dim in enumerate(names):
        print(f"{dim:28s}: {probs[i,1]:.3f} - {probs[i, 0]:.3f}")
        
    if show_graph:
        fig = plt.figure(figsize=(6,6))
        ax = fig.gca()
        
        xlabels = ["Introversion (I)", "Intuiting (N)", "Thinking (T)", "Judging (J)"]
        ax.barh(xlabels, [1, 1, 1, 1])
        ax.barh(xlabels, [ie[1], ns[1], tf[1], jp[1]])
        
        ax.set_xlim([0, 1])
        ax.set_xlabel("Propensity")
        
        plt.show(fig)

In [79]:
eval_string(about.text)

Introversion - Extroversion : 0.633 - 0.367
Intuiting - Sensing         : 0.603 - 0.397
Thinking - Feeling          : 0.807 - 0.193
Judging - Perceiving        : 0.734 - 0.266
